In [65]:
import json
import random

rows = []    
with open('botslookup.json') as data_file:
    bots = json.load(data_file)
    for bot in bots:
        rows.append(bot)

    
with open('nonbotslookup.json') as data_file:
    nonbots = json.load(data_file)
    for nonbot in nonbots:
        rows.append(nonbot)

realtests = []
with open('randomtests.json') as data_file:
    randomdata = json.load(data_file)
    for record in randomdata:
        realtests.append(record)

random.shuffle(rows)
random.shuffle(realtests)

In [66]:
from pandas import DataFrame
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
column_headers = ['default_profile', 'favourites_count', 'followers_count', 'friends_count', 'statuses_count', 'verified', 'retweet_count', 'class']
dataframe = DataFrame(np.array(rows), columns= column_headers)
array = dataframe.values
X = array[:,0:7]
Y = array[:,7]
data_train, data_test, target_train, target_test = train_test_split(X, Y, test_size=0.2, random_state=0)
num_trees = 100
max_features = 7
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
#Fit the training set
model.fit(data_train, target_train)

predicted_rf = model.predict(data_test)

In [67]:
#Precision, recall and F1
from sklearn.metrics import classification_report
target_names = ['bot', 'nonbot']
print(classification_report(target_test, predicted_rf, target_names=target_names))

#AUC 
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(target_test, predicted_rf, pos_label=1)
print(metrics.auc(fpr, tpr))

             precision    recall  f1-score   support

        bot       1.00      1.00      1.00        10
     nonbot       1.00      1.00      1.00        10

avg / total       1.00      1.00      1.00        20

1.0


In [73]:
realtestdataframe = DataFrame(np.array(realtests), columns=column_headers)
realarray = realtestdataframe.values

predicted_rf_real = model.predict(realarray[:,0:7])
np.mean(predicted_rf_real == realarray[:,7])

0.73333333333333328

In [75]:
#Precision, recall and F1
from sklearn.metrics import classification_report
target_names = ['bot', 'nonbot']
print(classification_report(realarray[:,7], predicted_rf_real, target_names=target_names))

#AUC 
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(realarray[:,7], predicted_rf_real, pos_label=1)
print(tpr)
print(fpr)
print(metrics.auc(fpr, tpr))

             precision    recall  f1-score   support

        bot       1.00      0.60      0.75        10
     nonbot       0.56      1.00      0.71         5

avg / total       0.85      0.73      0.74        15

[ 0.  1.  1.]
[ 0.   0.4  1. ]
0.8
